### Задание 1

    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv', sep = ',')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [54]:
# отфильтруем данные и выведем их в отдельный Df
# фильтруем данные --> группируем, считаем по каджому его количество и сбрасываем индекс в новом df
user_ratings = ratings[['userId', 'movieId']]
user_count_ratings = user_ratings.groupby('userId').count().reset_index()

# переименуем столбец 'movieId' в 'count_ratings' и отфильтруем его с количеством более 100
user_count_ratings.rename(columns={'movieId': 'count_ratings'}, inplace=True)
user_count_ratings = user_count_ratings.query('count_ratings > 100')
user_count_ratings

,userId,count_ratings
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423
...,...,...
655,656,128
658,659,142
663,664,519
664,665,434


In [15]:
# производим группировку, посчитаем  min|max timestamp-а, сбросим индексы для 
# корректного отображения таблицы и создадим новый столбец с разницой tampstamp-ов и назовем его 'timelife'
timestamp = ratings.groupby('userId').agg([min, max]).reset_index()
timestamp['timelife'] = timestamp['timestamp']['max'] - timestamp['timestamp']['min']
timestamp_final = timestamp[['userId', 'timelife']]
timestamp_final.head()

,userId,timelife
,,
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101


In [16]:
# соединим две готовые таблицы через merge
user_count_ratings.merge(timestamp_final, how = 'left', on = 'userId')

C:\Users\Home\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,userId,count_ratings,"(timelife, )"
0,4,204,203560
1,8,116,85187
2,15,1700,471393496
3,17,363,8053
4,19,423,5282
...,...,...,...
253,656,128,3053
254,659,142,31609411
255,664,519,98180439
256,665,434,54131471


In [96]:
# последним этапом просчитаем среднюю и выведем результат на экран

round_ = round(timestamp['timelife'].mean()/(60*60*24))
print(f'Cреднее время жизни пользователей, которые выставили более 100 оценок: {round_}')
timestamp.agg({'timelife': ['min', 'max', 'mean', 'median']}).round()

Cреднее время жизни пользователей, которые выставили более 100 оценок: 201


,timelife
min,29.0
max,515789661.0
mean,17373661.0
median,4360.0


### Задание 2

    Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [22]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [23]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [24]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [36]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base_id = client_base[['client_id']]
client_base_full = client_base

In [37]:
# без адреса
(
client_base_id.merge(rzd, how = 'outer', on = 'client_id')
    .merge(auto, how = 'outer', on = 'client_id')
    .merge(air, how = 'outer', on = 'client_id')
)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [38]:
# с адресом
(
client_base_full.merge(rzd, how = 'outer', on = 'client_id')
    .merge(auto, how = 'outer', on = 'client_id')
    .merge(air, how = 'outer', on = 'client_id')
)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


Задание 3

    В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

    У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
    Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
    Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Не совсем понял задание, но попробую дать ответ.
    1. Для связки координат с общими данными можно использовать метод merge по ID пользователя. Предполагаю, что для разделения покупок и визитов с координатами нам потребуются данные timestamp, т.е. где и в какой в момент совершения действия находился пользователь. Благодаря timestamp (timestamp визита|timestamp покупки) и ID можно корректно связать все данные для последующего анализа.
    2. Не понятен вопрос
    3. По координатам пользователей можно узнать, в каких регионах имеется повышенный спрос (или наоборот - низкий спрос) на продукцию компании, как можно оптимизировать логистику и направлении организации, если это, допустим, интернет-магазин. Возможно, где-то необходимо открыть|закрыть офисы продаж, перенаправить тот или иной товар в другой регион.